<a href="https://colab.research.google.com/github/ponchorojo/MVP2_PosPuc/blob/main/notebook_emprestimo_mvp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVP Pós graduação em Engenharia de Software
**Disciplina: Qualidade de Software, Segurança e Sistemas Inteligentes**

 **Projeto de Machine Learning: Aprovação de Empréstimos**

Treino de um modelo de classificação para prever se um empréstimo bancário será aprovado com base em dados históricos.

In [1]:
# Instalando bibliotecas com versões específicas para compatibilidade
!pip install numpy==1.24.4 scikit-learn==1.3.2 joblib==1.3.2

# Importando bibliotecas
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import joblib

### Carga do dataset, limpeza e separação de dados
Utilizado dataset público sobre aprovação de empréstimos, carregado diretamente de uma URL para garantir reprodutibilidade.
Selecionamos apenas as colunas relevantes para prever se o empréstimo será aprovado ou não e transformação da variável (`Loan_Status`) para valores binários (1 para 'Y' e 0 para 'N').

In [2]:
# 2. Carregando dataset via URL
url = 'https://raw.githubusercontent.com/naveen-chauhan/Loan-Prediction-Classification/master/Dataset/train.csv'
df = pd.read_csv(url)
df = df[['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History', 'Loan_Status']]
df.dropna(inplace=True)
df['Loan_Status'] = df['Loan_Status'].map({'Y': 1, 'N': 0})
df.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status
1,4583,1508.0,128.0,360.0,1.0,0
2,3000,0.0,66.0,360.0,1.0,1
3,2583,2358.0,120.0,360.0,1.0,1
4,6000,0.0,141.0,360.0,1.0,1
5,5417,4196.0,267.0,360.0,1.0,1


### Separação entre treino e teste (holdout)
Aplicada a técnica de holdout, separando 80% dos dados para treino e 20% para teste. Essa separação permite avaliar se o modelo consegue generalizar bem para dados que ele nunca viu.

In [3]:
# Separação treino e teste
X = df.drop('Loan_Status', axis=1)
y = df['Loan_Status']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Salvando dados de teste para PyTest
df_test = X_test.copy()
df_test['Loan_Status'] = y_test.values
df_test.to_csv('dados_teste.csv', index=False)

### Modelagem com KNN, Árvore de Decisão, Naive Bayes e SVM
Utilizado 4 algoritmos de classificação: KNN, Árvore de Decisão, Naive Bayes e SVM de forma a realizar comparações de desempenho entre cada modelo.
Cada modelo possui etapas de pré-processamento e padronização, seguidas pelo  treinamento.


In [4]:
# 4. Treinando e avaliando modelos
pipelines = {
    'KNN': Pipeline([('imp', SimpleImputer()), ('scaler', StandardScaler()), ('model', KNeighborsClassifier())]),
    'Tree': Pipeline([('imp', SimpleImputer()), ('model', DecisionTreeClassifier())]),
    'NB': Pipeline([('imp', SimpleImputer()), ('model', GaussianNB())]),
    'SVM': Pipeline([('imp', SimpleImputer()), ('scaler', StandardScaler()), ('model', SVC())])
}

melhor_modelo = None
melhor_score = 0

for nome, pipe in pipelines.items():
    pipe.fit(X_train, y_train)
    score = pipe.score(X_test, y_test)
    print(f"{nome} - Acurácia: {score:.2f}")
    if score > melhor_score:
        melhor_score = score
        melhor_modelo = pipe

print("\nRelatório do melhor modelo:")
y_pred = melhor_modelo.predict(X_test)
print(classification_report(y_test, y_pred))

KNN - Acurácia: 0.79
Tree - Acurácia: 0.74
NB - Acurácia: 0.83
SVM - Acurácia: 0.83

Relatório do melhor modelo:
              precision    recall  f1-score   support

           0       0.86      0.55      0.67        33
           1       0.82      0.96      0.89        73

    accuracy                           0.83       106
   macro avg       0.84      0.75      0.78       106
weighted avg       0.83      0.83      0.82       106



### Cross-Validation

Ajuda a detectar se algum modelo está sofrendo *overfitting*.

In [5]:
# 6. Validação cruzada
from sklearn.model_selection import cross_val_score

for nome, pipe in pipelines.items():
    scores = cross_val_score(pipe, X, y, cv=5, scoring='accuracy')
    print(f"{nome} - Acurácia (CV): {scores.mean():.2f} (+/- {scores.std():.2f})")

KNN - Acurácia (CV): 0.78 (+/- 0.02)
Tree - Acurácia (CV): 0.74 (+/- 0.03)
NB - Acurácia (CV): 0.80 (+/- 0.03)
SVM - Acurácia (CV): 0.81 (+/- 0.03)


### Otimização de hiperparâmetros e avaliação final do modelo no conjunto de teste

Para maximizar a acurácia média em validação cruzada, garantindo melhor desempenho geral.

Avaliação do modelo no conjunto de teste com métricas de classificação: acurácia, precisão, recall e F1-score.
de forma a entender o equilíbrio entre falsos positivos e falsos negativos.

In [6]:
# Otimização de hiperparâmetros
from sklearn.model_selection import GridSearchCV

param_grid = {
    'model__C': [0.1, 1, 10],
    'model__kernel': ['linear', 'rbf']
}

pipe_svm = Pipeline([
    ('imp', SimpleImputer()),
    ('scaler', StandardScaler()),
    ('model', SVC())
])

grid = GridSearchCV(pipe_svm, param_grid, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)

print(f"Melhor acurácia (GridSearchCV): {grid.best_score_:.2f}")
print("Melhores hiperparâmetros:", grid.best_params_)

# Avaliação no conjunto de teste com o melhor estimador
melhor_modelo = grid.best_estimator_
y_pred = melhor_modelo.predict(X_test)
print(classification_report(y_test, y_pred))

Melhor acurácia (GridSearchCV): 0.81
Melhores hiperparâmetros: {'model__C': 0.1, 'model__kernel': 'linear'}
              precision    recall  f1-score   support

           0       0.94      0.48      0.64        33
           1       0.81      0.99      0.89        73

    accuracy                           0.83       106
   macro avg       0.88      0.74      0.76       106
weighted avg       0.85      0.83      0.81       106



In [7]:
# Salvando o modelo
joblib.dump(melhor_modelo, 'modelo_treinado.pkl')

# Download do modelo
from google.colab import files
files.download('modelo_treinado.pkl')

# Download dos dados de teste
from google.colab import files
files.download('dados_teste.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Teste das métricas do modelo final

In [8]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Thresholds configuráveis
min_precision = 0.70
min_recall = 0.70
min_f1 = 0.70

# Avaliação do modelo
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")

assert precision >= min_precision, f"Precision abaixo do mínimo: {precision:.2f}"
assert recall >= min_recall, f"Recall abaixo do mínimo: {recall:.2f}"
assert f1 >= min_f1, f"F1-score abaixo do mínimo: {f1:.2f}"

print("Modelo atende aos requisitos mínimos de desempenho.")

Precision: 0.81
Recall: 0.99
F1-score: 0.89
Modelo atende aos requisitos mínimos de desempenho.


## Conclusão final

Após testar os quatro modelos clássicos — KNN, Árvore de Decisão, Naive Bayes e SVM — percebi que o desempenho variou significativamente entre eles. O SVM, especialmente após otimização, se destacou com os melhores resultados gerais nas métricas de avaliação.

Com validação cruzada, ele atingiu uma média de acurácia acima de 75%, o que me deu mais confiança sobre sua capacidade de generalização. Além da acurácia, as métricas de precisão, recall e F1-score também ficaram acima do mínimo estabelecido de 70%, o que reforça o equilíbrio entre falsos positivos e falsos negativos.

Apesar disso, vale destacar que o dataset é relativamente pequeno e desbalanceado, o que pode impactar a qualidade do aprendizado. Mesmo assim, os resultados foram satisfatórios considerando o escopo do projeto.